In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [3]:
dfmov = pd.read_csv('../../data/imdb_movies_clean_1st.csv')

In [4]:
dfmov.head(1)

,Unnamed: 0,imdb_title_id,original_title,year,genre,duration,country,language,director,writer,production_company,actors,description
0,0,tt0000009,Miss Jerry,1894,Romance,45,USA,0,Alexander Black,Alexander Black,Alexander Black Photoplays,"Blanche Bayliss, William Courtenay, Chauncey D...",The adventures of a female reporter in the 1890s.


In [5]:
dfrat = pd.read_csv('../../data/IMDb_ratings.csv')

In [11]:
dfrat.head(5)

,imdb_title_id,weighted_average_vote,total_votes,mean_vote,median_vote,votes_10,votes_9,votes_8,votes_7,votes_6,...,females_30age_avg_vote,females_30age_votes,females_45age_avg_vote,females_45age_votes,top1000_voters_rating,top1000_voters_votes,us_voters_rating,us_voters_votes,non_us_voters_rating,non_us_voters_votes
0,tt0000009,5.9,154,5.9,6.0,12,4,10,43,28,...,5.7,13.0,4.5,4.0,5.7,34.0,6.4,51.0,6.0,70.0
1,tt0000574,6.1,589,6.3,6.0,57,18,58,137,139,...,6.2,23.0,6.6,14.0,6.4,66.0,6.0,96.0,6.2,331.0
2,tt0001892,5.8,188,6.0,6.0,6,6,17,44,52,...,5.8,4.0,6.8,7.0,5.4,32.0,6.2,31.0,5.9,123.0
3,tt0002101,5.2,446,5.3,5.0,15,8,16,62,98,...,5.5,14.0,6.1,21.0,4.9,57.0,5.5,207.0,4.7,105.0
4,tt0002130,7.0,2237,6.9,7.0,210,225,436,641,344,...,7.3,82.0,7.4,77.0,6.9,139.0,7.0,488.0,7.0,1166.0


In [7]:
dfrat.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 85855 entries, 0 to 85854
Data columns (total 49 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   imdb_title_id              85855 non-null  object 
 1   weighted_average_vote      85855 non-null  float64
 2   total_votes                85855 non-null  int64  
 3   mean_vote                  85855 non-null  float64
 4   median_vote                85855 non-null  float64
 5   votes_10                   85855 non-null  int64  
 6   votes_9                    85855 non-null  int64  
 7   votes_8                    85855 non-null  int64  
 8   votes_7                    85855 non-null  int64  
 9   votes_6                    85855 non-null  int64  
 10  votes_5                    85855 non-null  int64  
 11  votes_4                    85855 non-null  int64  
 12  votes_3                    85855 non-null  int64  
 13  votes_2                    85855 non-null  int

This is an example of collaborative filtering through user. Meaning it will use other users information to recommend articles to the user in place. It tries finding users with similar preferences and opinions. The used method here is the Pearson corelation.

In [8]:
# Muy probable que vaya que utilizar la media de muchos valores que ya la tienes hecha

Using a rating as a metric has a few caveats:
- For one, it does not take into consideration the popularity of a movie. Therefore, a movie with a rating of 9 from 10 voters will be considered 'better' than a movie with a rating of 8.9 from 10,000 voters.

- This metric will also tend to favor movies with a smaller number of voters with skewed and/or extremely high ratings. As the number of voters increases, the rating of a movie regularizes and approaches towards a value that is reflective of the movie's quality and gives the user a much better idea as to which movie he/she should choose. External sources might be necessary to provide proper recommendations.

Taking these shortcomings into consideration, you must come up with a weighted rating that takes into account the average rating and the number of votes it has accumulated. Such a system will make sure that a movie with a 9 rating from 100,000 voters gets a (far) higher score than a movie with the same rating but a mere few hundred voters.

## Simple Recommender (Top voted films)

In [12]:
# Mean of vote average column
tot_mean = dfrat['weighted_average_vote'].mean()
print(tot_mean)

5.898655873274613


In [13]:
# Minimun number of votes requireds to be in chart, I am going to give scores of 80%+
min_votes = dfrat['total_votes'].quantile(0.80)
print(min_votes)

2722.0


In [14]:
q_movies = dfrat.copy().loc[dfrat['total_votes']>= min_votes]

In [17]:
q_movies

,imdb_title_id,weighted_average_vote,total_votes,mean_vote,median_vote,votes_10,votes_9,votes_8,votes_7,votes_6,...,females_30age_avg_vote,females_30age_votes,females_45age_avg_vote,females_45age_votes,top1000_voters_rating,top1000_voters_votes,us_voters_rating,us_voters_votes,non_us_voters_rating,non_us_voters_votes
24,tt0003740,7.1,3073,6.5,7.0,285,301,591,727,443,...,6.9,134.0,6.8,194.0,6.9,177.0,7.0,1035.0,7.0,1332.0
39,tt0004707,6.3,3250,7.1,7.0,838,100,312,685,643,...,6.6,174.0,6.7,402.0,6.1,185.0,6.4,1544.0,6.2,1056.0
47,tt0004972,6.3,22213,6.4,7.0,3661,1741,3314,3963,2876,...,5.8,1106.0,5.9,725.0,6.5,355.0,6.3,7452.0,6.4,8306.0
61,tt0006206,7.3,4166,6.7,7.0,620,439,763,872,484,...,7.1,256.0,7.7,306.0,6.8,204.0,7.5,1350.0,7.2,1945.0
76,tt0006864,7.8,13875,7.8,8.0,3477,2230,3214,2249,1179,...,7.4,660.0,8.0,662.0,7.5,321.0,7.7,4286.0,7.8,5954.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85780,tt9817070,7.9,5658,8.1,8.0,1430,1032,1604,845,358,...,7.7,161.0,7.6,27.0,6.3,48.0,7.5,236.0,7.7,1987.0
85811,tt9860728,5.6,14108,5.7,6.0,769,420,1102,2332,3613,...,5.7,2363.0,6.0,500.0,4.7,91.0,5.8,1498.0,5.5,5481.0
85837,tt9894470,6.1,4178,6.1,6.0,189,151,495,1020,1069,...,5.7,104.0,5.3,39.0,5.5,104.0,6.4,847.0,6.0,1907.0
85839,tt9898858,5.1,10627,5.1,5.0,387,197,560,1398,2392,...,5.0,537.0,5.0,163.0,4.5,99.0,5.1,1562.0,5.0,3903.0


In [23]:
pd.set_option('display.max_columns', None)
q_movies.head()

,imdb_title_id,weighted_average_vote,total_votes,mean_vote,median_vote,votes_10,votes_9,votes_8,votes_7,votes_6,votes_5,votes_4,votes_3,votes_2,votes_1,allgenders_0age_avg_vote,allgenders_0age_votes,allgenders_18age_avg_vote,allgenders_18age_votes,allgenders_30age_avg_vote,allgenders_30age_votes,allgenders_45age_avg_vote,allgenders_45age_votes,males_allages_avg_vote,males_allages_votes,males_0age_avg_vote,males_0age_votes,males_18age_avg_vote,males_18age_votes,males_30age_avg_vote,males_30age_votes,males_45age_avg_vote,males_45age_votes,females_allages_avg_vote,females_allages_votes,females_0age_avg_vote,females_0age_votes,females_18age_avg_vote,females_18age_votes,females_30age_avg_vote,females_30age_votes,females_45age_avg_vote,females_45age_votes,top1000_voters_rating,top1000_voters_votes,us_voters_rating,us_voters_votes,non_us_voters_rating,non_us_voters_votes
24,tt0003740,7.1,3073,6.5,7.0,285,301,591,727,443,199,85,27,18,397,6.0,3.0,7.0,393.0,7.0,1126.0,7.2,1006.0,7.1,2149.0,6.0,2.0,7.0,323.0,7.0,976.0,7.3,799.0,6.9,402.0,6.0,1.0,6.8,67.0,6.9,134.0,6.8,194.0,6.9,177.0,7.0,1035.0,7.0,1332.0
39,tt0004707,6.3,3250,7.1,7.0,838,100,312,685,643,334,146,92,53,47,5.0,1.0,6.3,227.0,6.2,1090.0,6.4,1392.0,6.3,2107.0,5.0,1.0,6.3,183.0,6.2,909.0,6.4,978.0,6.6,622.0,NaN,NaN,6.4,41.0,6.6,174.0,6.7,402.0,6.1,185.0,6.4,1544.0,6.2,1056.0
47,tt0004972,6.3,22213,6.4,7.0,3661,1741,3314,3963,2876,1928,978,701,577,2474,4.7,14.0,6.0,3183.0,6.3,8861.0,6.7,4901.0,6.4,14818.0,4.7,12.0,6.1,2623.0,6.4,7669.0,6.8,4104.0,5.7,2417.0,6.0,1.0,5.4,517.0,5.8,1106.0,5.9,725.0,6.5,355.0,6.3,7452.0,6.4,8306.0
61,tt0006206,7.3,4166,6.7,7.0,620,439,763,872,484,232,140,40,31,545,6.5,7.0,7.1,517.0,7.3,1661.0,7.3,1294.0,7.3,2835.0,6.8,6.0,7.1,414.0,7.3,1384.0,7.3,976.0,7.2,670.0,4.0,1.0,6.9,98.0,7.1,256.0,7.7,306.0,6.8,204.0,7.5,1350.0,7.2,1945.0
76,tt0006864,7.8,13875,7.8,8.0,3477,2230,3214,2249,1179,605,340,181,133,267,7.9,8.0,7.8,1795.0,7.7,5451.0,7.8,3667.0,7.8,9441.0,8.0,7.0,7.9,1498.0,7.7,4734.0,7.8,2970.0,7.5,1632.0,7.0,1.0,7.2,276.0,7.4,660.0,8.0,662.0,7.5,321.0,7.7,4286.0,7.8,5954.0


In [15]:
# My new created dataframe with films with number of votes over 2722
q_movies.shape
# We can clearly see how 17k.. is around 20% of 85k..

(17172, 49)

In [16]:
dfrat.shape

(85855, 49)

In [19]:
def weighted_rating(x, min_votes=min_votes, tot_mean=tot_mean):
    '''
    This function computes the weighted average of each movie
    '''
    vc = x['total_votes']
    vr = x['weighted_average_vote']
    
    return (vc/(vc+min_votes)*vr) + (min_votes/(min_votes+vc)*tot_mean)

In [24]:
# I define a new feature, 'score' for will I will calculate its value with the created function.

q_movies['score'] = q_movies.apply(weighted_rating, axis=1)

In [36]:
# Sort movies based on the score they got above

q_movies = q_movies.sort_values('score', ascending=False)

In [37]:
q_movies

,imdb_title_id,weighted_average_vote,total_votes,mean_vote,median_vote,votes_10,votes_9,votes_8,votes_7,votes_6,votes_5,votes_4,votes_3,votes_2,votes_1,allgenders_0age_avg_vote,allgenders_0age_votes,allgenders_18age_avg_vote,allgenders_18age_votes,allgenders_30age_avg_vote,allgenders_30age_votes,allgenders_45age_avg_vote,allgenders_45age_votes,males_allages_avg_vote,males_allages_votes,males_0age_avg_vote,males_0age_votes,males_18age_avg_vote,males_18age_votes,males_30age_avg_vote,males_30age_votes,males_45age_avg_vote,males_45age_votes,females_allages_avg_vote,females_allages_votes,females_0age_avg_vote,females_0age_votes,females_18age_avg_vote,females_18age_votes,females_30age_avg_vote,females_30age_votes,females_45age_avg_vote,females_45age_votes,top1000_voters_rating,top1000_voters_votes,us_voters_rating,us_voters_votes,non_us_voters_rating,non_us_voters_votes,score
28453,tt0111161,9.3,2278845,9.1,10.0,1259076,582166,264604,84779,25558,13167,6419,4615,4303,34158,9.3,1898.0,9.3,489526.0,9.3,863727.0,9.1,204278.0,9.3,1392803.0,9.3,1327.0,9.3,389793.0,9.3,721951.0,9.1,165852.0,9.2,274168.0,9.0,267.0,9.2,88528.0,9.2,129249.0,9.2,34317.0,8.9,906.0,9.3,327264.0,9.2,887226.0,9.295942
15528,tt0068646,9.2,1572674,8.9,10.0,813223,389002,192260,76045,29201,16157,8222,6120,5790,36654,9.2,1208.0,9.2,312132.0,9.2,619447.0,9.1,159410.0,9.2,1004808.0,9.2,869.0,9.3,260811.0,9.2,537010.0,9.2,133330.0,8.9,151729.0,8.9,156.0,8.9,44300.0,8.9,73713.0,8.9,22915.0,9.1,889.0,9.2,245541.0,9.2,633246.0,9.194296
38406,tt0252487,9.3,36269,9.3,10.0,26886,4619,2062,982,357,199,98,90,103,873,8.6,55.0,9.4,9499.0,9.3,15399.0,8.2,1653.0,9.3,24960.0,9.0,41.0,9.4,8309.0,9.3,13589.0,8.2,1448.0,9.3,3097.0,7.4,11.0,9.3,1043.0,9.4,1612.0,8.0,183.0,5.7,101.0,7.9,1025.0,9.3,18627.0,9.062549
48078,tt0468569,9.0,2241615,8.9,9.0,1006449,629582,344047,134153,48430,22713,11230,7942,7063,30006,9.2,2097.0,9.2,522182.0,9.0,879620.0,8.4,145726.0,9.1,1409165.0,9.2,1544.0,9.3,422587.0,9.0,743676.0,8.4,122761.0,8.7,250634.0,8.8,224.0,8.8,88034.0,8.7,123900.0,8.2,20125.0,8.6,928.0,9.1,348363.0,8.9,861033.0,8.996239
16556,tt0071562,9.0,1098714,8.8,9.0,472758,314803,170068,68732,25674,12350,6069,4261,3832,20167,9.1,721.0,9.0,208613.0,9.0,435756.0,8.9,118888.0,9.0,712572.0,9.1,532.0,9.1,178943.0,9.0,381914.0,9.0,100306.0,8.7,96023.0,9.0,103.0,8.8,25412.0,8.7,47656.0,8.7,16306.0,8.8,856.0,9.1,164758.0,9.0,448555.0,8.992336
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55494,tt1213644,1.9,86219,2.5,1.0,6490,945,829,1214,1922,2951,3540,5603,10033,52692,1.3,48.0,2.0,23480.0,1.8,40258.0,2.0,5806.0,1.9,62606.0,1.1,39.0,2.0,19257.0,1.8,35732.0,2.0,5204.0,2.2,9099.0,3.3,5.0,2.3,3924.0,2.0,4248.0,2.1,535.0,2.1,413.0,1.6,18243.0,1.9,42205.0,2.022377
74733,tt4458206,1.3,27100,2.5,1.0,4092,104,68,54,63,74,65,105,239,22236,1.0,16.0,1.3,6353.0,1.2,9199.0,1.6,1284.0,1.2,16800.0,1.0,14.0,1.2,5701.0,1.2,8363.0,1.5,1141.0,1.4,1490.0,1.0,1.0,1.5,578.0,1.2,681.0,2.4,118.0,2.0,79.0,1.6,848.0,1.1,9638.0,1.719742
79280,tt5988370,1.4,72151,2.8,1.0,13844,114,58,52,40,66,38,56,216,57667,2.4,81.0,1.5,20954.0,1.2,19182.0,1.2,2924.0,1.4,46295.0,2.4,65.0,1.5,17431.0,1.2,16752.0,1.2,2522.0,1.5,6808.0,2.3,16.0,1.7,3346.0,1.1,2036.0,1.5,325.0,2.2,29.0,1.0,1456.0,1.4,31818.0,1.563548
82091,tt7221896,1.0,37592,2.0,1.0,3986,58,56,82,70,138,68,156,234,32744,3.1,254.0,1.1,12678.0,1.0,5474.0,1.6,752.0,1.0,24960.0,3.2,232.0,1.1,11698.0,1.0,5058.0,1.5,681.0,1.2,1765.0,1.7,18.0,1.2,865.0,1.2,301.0,3.3,64.0,1.9,25.0,1.1,378.0,1.1,14689.0,1.330757


In [38]:
dfmov.head()

,Unnamed: 0,imdb_title_id,original_title,year,genre,duration,country,language,director,writer,production_company,actors,description
0,0,tt0000009,Miss Jerry,1894,Romance,45,USA,0,Alexander Black,Alexander Black,Alexander Black Photoplays,"Blanche Bayliss, William Courtenay, Chauncey D...",The adventures of a female reporter in the 1890s.
1,1,tt0000574,The Story of the Kelly Gang,1906,"Biography, Crime, Drama",70,Australia,0,Charles Tait,Charles Tait,J. and N. Tait,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...
2,2,tt0001892,Den sorte drøm,1911,Drama,53,"Germany, Denmark",0,Urban Gad,"Urban Gad, Gebhard Schätzler-Perasini",Fotorama,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...
3,3,tt0002101,Cleopatra,1912,"Drama, History",100,USA,English,Charles L. Gaskill,Victorien Sardou,Helen Gardner Picture Players,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",The fabled queen of Egypt's affair with Roman ...
4,4,tt0002130,L'Inferno,1911,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",Dante Alighieri,Milano Film,"Salvatore Papa, Arturo Pirovano, Giuseppe de L...",Loosely adapted from Dante's Divine Comedy and...


In [39]:
q_movies[['imdb_title_id', 'weighted_average_vote', 'total_votes', 'score']].head(20)

,imdb_title_id,weighted_average_vote,total_votes,score
28453,tt0111161,9.3,2278845,9.295942
15528,tt0068646,9.2,1572674,9.194296
38406,tt0252487,9.3,36269,9.062549
48078,tt0468569,9.0,2241615,8.996239
16556,tt0071562,9.0,1098714,8.992336
28381,tt0110912,8.9,1780147,8.895418
34127,tt0167260,8.9,1604280,8.894916
27629,tt0108052,8.9,1183248,8.893111
8973,tt0050083,8.9,668473,8.887828
57475,tt1375666,8.8,2002816,8.796062


In [40]:
top_selection = pd.merge(dfmov,  q_movies, on='imdb_title_id', how='left')

In [41]:
top_selection

,Unnamed: 0,imdb_title_id,original_title,year,genre,duration,country,language,director,writer,production_company,actors,description,weighted_average_vote,total_votes,mean_vote,median_vote,votes_10,votes_9,votes_8,votes_7,votes_6,votes_5,votes_4,votes_3,votes_2,votes_1,allgenders_0age_avg_vote,allgenders_0age_votes,allgenders_18age_avg_vote,allgenders_18age_votes,allgenders_30age_avg_vote,allgenders_30age_votes,allgenders_45age_avg_vote,allgenders_45age_votes,males_allages_avg_vote,males_allages_votes,males_0age_avg_vote,males_0age_votes,males_18age_avg_vote,males_18age_votes,males_30age_avg_vote,males_30age_votes,males_45age_avg_vote,males_45age_votes,females_allages_avg_vote,females_allages_votes,females_0age_avg_vote,females_0age_votes,females_18age_avg_vote,females_18age_votes,females_30age_avg_vote,females_30age_votes,females_45age_avg_vote,females_45age_votes,top1000_voters_rating,top1000_voters_votes,us_voters_rating,us_voters_votes,non_us_voters_rating,non_us_voters_votes,score
0,0,tt0000009,Miss Jerry,1894,Romance,45,USA,0,Alexander Black,Alexander Black,Alexander Black Photoplays,"Blanche Bayliss, William Courtenay, Chauncey D...",The adventures of a female reporter in the 1890s.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,tt0000574,The Story of the Kelly Gang,1906,"Biography, Crime, Drama",70,Australia,0,Charles Tait,Charles Tait,J. and N. Tait,"Elizabeth Tait, John Tait, Norman Campbell, Be...",True story of notorious Australian outlaw Ned ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,tt0001892,Den sorte drøm,1911,Drama,53,"Germany, Denmark",0,Urban Gad,"Urban Gad, Gebhard Schätzler-Perasini",Fotorama,"Asta Nielsen, Valdemar Psilander, Gunnar Helse...",Two men of high rank are both wooing the beaut...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,3,tt0002101,Cleopatra,1912,"Drama, History",100,USA,English,Charles L. Gaskill,Victorien Sardou,Helen Gardner Picture Players,"Helen Gardner, Pearl Sindelar, Miss Fielding, ...",The fabled queen of Egypt's affair with Roman ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,4,tt0002130,L'Inferno,1911,"Adventure, Drama, Fantasy",68,Italy,Italian,"Francesco Bertolini, Adolfo Padovan",Dante Alighieri,Milano Film,"Salvatore Papa, Arturo Pirovano, Giuseppe de L...",Loosely adapted from Dante's Divine Comedy and...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
85850,85850,tt9908390,Le lion,2020,Comedy,95,"France, Belgium",French,Ludovic Colbeau-Justin,"Alexandre Coquelle, Matthieu Le Naour",Monkey Pack Films,"Dany Boon, Philippe Katerine, Anne Serra, Samu...",A psychiatric hospital patient pretends to be ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
85851,85851,tt9911196,De Beentjes van Sint-Hildegard,2020,"Comedy, Drama",103,Netherlands,"German, Dutch",Johan Nijenhuis,"Radek Bajgar, Herman Finkers",Johan Nijenhuis & Co,"Herman F

In [42]:
top_selection.isnull().sum()

Unnamed: 0                  0
imdb_title_id               0
original_title              0
year                        0
genre                       0
                        ...  
us_voters_rating        68683
us_voters_votes         68683
non_us_voters_rating    68683
non_us_voters_votes     68683
score                   68683
Length: 62, dtype: int64

In [43]:
top_selection = top_selection.sort_values('score', ascending=False)

In [45]:
top_selection.head(20)
# This is the top 20 selection based on my scrore calculation of votes 

,Unnamed: 0,imdb_title_id,original_title,year,genre,duration,country,language,director,writer,production_company,actors,description,weighted_average_vote,total_votes,mean_vote,median_vote,votes_10,votes_9,votes_8,votes_7,votes_6,votes_5,votes_4,votes_3,votes_2,votes_1,allgenders_0age_avg_vote,allgenders_0age_votes,allgenders_18age_avg_vote,allgenders_18age_votes,allgenders_30age_avg_vote,allgenders_30age_votes,allgenders_45age_avg_vote,allgenders_45age_votes,males_allages_avg_vote,males_allages_votes,males_0age_avg_vote,males_0age_votes,males_18age_avg_vote,males_18age_votes,males_30age_avg_vote,males_30age_votes,males_45age_avg_vote,males_45age_votes,females_allages_avg_vote,females_allages_votes,females_0age_avg_vote,females_0age_votes,females_18age_avg_vote,females_18age_votes,females_30age_avg_vote,females_30age_votes,females_45age_avg_vote,females_45age_votes,top1000_voters_rating,top1000_voters_votes,us_voters_rating,us_voters_votes,non_us_voters_rating,non_us_voters_votes,score
28453,28453,tt0111161,The Shawshank Redemption,1994,Drama,142,USA,English,Frank Darabont,"Stephen King, Frank Darabont",Castle Rock Entertainment,"Tim Robbins, Morgan Freeman, Bob Gunton, Willi...",Two imprisoned men bond over a number of years...,9.3,2278845.0,9.1,10.0,1259076.0,582166.0,264604.0,84779.0,25558.0,13167.0,6419.0,4615.0,4303.0,34158.0,9.3,1898.0,9.3,489526.0,9.3,863727.0,9.1,204278.0,9.3,1392803.0,9.3,1327.0,9.3,389793.0,9.3,721951.0,9.1,165852.0,9.2,274168.0,9.0,267.0,9.2,88528.0,9.2,129249.0,9.2,34317.0,8.9,906.0,9.3,327264.0,9.2,887226.0,9.295942
15528,15528,tt0068646,The Godfather,1972,"Crime, Drama",175,USA,"English, Italian, Latin",Francis Ford Coppola,"Mario Puzo, Francis Ford Coppola",Paramount Pictures,"Marlon Brando, Al Pacino, James Caan, Richard ...",The aging patriarch of an organized crime dyna...,9.2,1572674.0,8.9,10.0,813223.0,389002.0,192260.0,76045.0,29201.0,16157.0,8222.0,6120.0,5790.0,36654.0,9.2,1208.0,9.2,312132.0,9.2,619447.0,9.1,159410.0,9.2,1004808.0,9.2,869.0,9.3,260811.0,9.2,537010.0,9.2,133330.0,8.9,151729.0,8.9,156.0,8.9,44300.0,8.9,73713.0,8.9,22915.0,9.1,889.0,9.2,245541.0,9.2,633246.0,9.194296
38406,38406,tt0252487,Hababam Sinifi,1975,"Comedy, Drama",87,Turkey,Turkish,Ertem Egilmez,"Umur Bugay, Rifat Ilgaz",Arzu Film,"Kemal Sunal, Münir Özkul, Halit Akçatepe, Tari...","Lazy, uneducated students share a very close b...",9.3,36269.0,9.3,10.0,26886.0,4619.0,2062.0,982.0,357.0,199.0,98.0,90.0,103.0,873.0,8.6,55.0,9.4,9499.0,9.3,15399.0,8.2,1653.0,9.3,24960.0,9.0,41.0,9.4,8309.0,9.3,13589.0,8.2,1448.0,9.3,3097.0,7.4,11.0,9.3,1043.0,9.4,1612.0,8.0,183.0,5.7,101.0,7.9,1025.0,9.3,18627.0,9.062549
48078,48078,tt0468569,The Dark Knight,2008,"Action, Crime, Drama",152,"USA, UK","English, Mandarin",Christopher Nolan,"Jonathan Nolan, Christopher Nolan",Warner Bros.,"Christian Bale, Heath Ledger, Aaron Eckhart, M...",When the menace known as the Joker wreaks havo...,9.0,2241615.0,8.9,9.0,1006449.0,629582.0,344047.0,134153.0,48430.0,22713.0,11230.0,7942.0,7063.0,30006.0,9.2,2097.0,9.2,522182.0,9.0,879620.0,8.4,145726.0,9.1,1409165.0,9.2,1544.0,9.3,422587.0,9.0,743676.0,8.4,122761.0,8.7,250634.0,8.8,224.0,8.8,88034.0,8.7,123900.0,8.2,20125.0,8.6,928.0,9.1,348363.0,8.9,861033.0,8.996239
16556,16556,tt0071562,The Godfather: Part II,1974,"Crime, Drama",202,USA,"English, Italian, Spanish, Latin, Sicilian",Francis Ford Coppola,"Francis Ford Coppola, Mario Puzo",Paramount Pictures,"Al Pacino, Robert Duvall, Diane Keaton, Robert...",The early life and career of Vito Corleone in ...,9.0,1098714.0,8.8,9.0,472758.0,314803.0,170068.0,68732.0,25674.0,12350.0,6069.0,4261.0,3832.0,20167.0,9.1,721.0,9.0,208613.0,9.0,435756.0,8.9,118888.0,9.0,712572.0,9.1,532.0,9.1,178943.0,9.0,381914.0,9.0,100306.0,8.7,96023.0,9.0,103.0,8.8,25412.0,8.7,47656.0,8.7,16306.0,8.8,856.0,9.1,164758.0,9.0,448555.0,8.992336
28381,28381,tt0110912,Pulp Fiction,1994,"Crime, Drama",154,USA,"English, Spanish, French",Quentin Tarantino,"Quentin Tarantino, Roger